In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

>  the data path

In [ ]:
data = '/kaggle/input/hepatitis-c-dataset/HepatitisCdata.csv'
df = pd.read_csv(data, index_col=0)

# Showing number of patients 

print(df.index[df['Category'] == '0=Blood Donor'].size)
print(df.index[df['Category'] == '0s=suspect Blood Donor'].size)
print(df.index[df['Category'] == '1=Hepatitis'].size)
print(df.index[df['Category'] == '2=Fibrosis'].size)
print(df.index[df['Category'] == '3=Cirrhosis'].size)

 There is not clarification towards the goal of the model; Classify in all 4 potential cases, or just distinguish between blood donor and infected person???

In [ ]:
print(df.head(), df.describe())

Some empty values exist they will be filled by the mean.

In [ ]:
 df.fillna(df.mean(), inplace = True)
df.describe()

In [ ]:
df.info()
print(df.shape)

We will change all the datatypes to float64

In [ ]:
#πρεπει τα 4 categories να γινουν 0,1,2,3

df['Category'].replace({"0=Blood Donor" : "0", "0s=suspect Blood Donor" : "0.5", "1=Hepatitis" : "1", "2=Fibrosis" : "2", "3=Cirrhosis" : "3"}, inplace=True)
df['Category'].astype(float)
df['Age'].astype(float)
df['Sex'].replace({"m" : "0", "f" : "1"}, inplace = True)
df['Sex'].astype(float)

In [ ]:
#Is it more preferable to falsly diagnose a healthy patient with hepatitis instead of diagnosing them as healthy while being sick?
#AKA what is better; false positive or false negative?
#I will follow the false positive path on the grounds that; By assigning a patient as sick they will proceed to medical attention hence the condition will not worsen.
df['Category'].replace(0.5 , 1, inplace=True)

print(df['Category'].unique()) #0.5 is still been seen as a unique 
print(df.index[df['Category'] == 0.5].size) #even though it has been replaced
print(df['Category'].nunique())

In [ ]:
from sklearn.utils import shuffle
df = shuffle(df)
df.head()

# Multinomial Logistic Regression

In [ ]:
import sklearn.model_selection
X = df.drop(columns='Category')
y = df.Category
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X, y, test_size = 0.20, random_state = 5)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
from sklearn.linear_model import LogisticRegression

model1 = LogisticRegression(random_state=0, multi_class='multinomial', penalty='none', solver='newton-cg').fit(X_train, y_train)
preds = model1.predict(X_test)

In [ ]:
params = model1.get_params()
print(params)

In [ ]:
print('Intercept: \n', model1.intercept_)
print('Coefficients: \n', model1.coef_)

In [ ]:
np.exp(model1.coef_)

# Accuracy report

In [ ]:
from sklearn.metrics import classification_report
from sklearn import metrics 
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, preds)

In [ ]:
confmtrx = np.array(confusion_matrix(y_test, preds))
print(df['Category'].nunique())

pd.DataFrame(confmtrx, index=['Blood Donors','Suspected Donors', 'Hepatitis', 'Fibrosis', 'Cirrhosis' ],
columns=['predicted_Healthy',  'predicted_Hepatitis', 'predicted_suspected', 'predicted_Fibrosis', 'predicted_Cirrhosis'])

In [ ]:
#Acccuracy stats
print('Accuracy Score:', metrics.accuracy_score(y_test, preds)) 

#Classification report
class_report=classification_report(y_test, preds)
print(class_report)

1. Macro average; averaging the unweighted mean per label
2. Weighted average; averaging the support-weighted mean per label
3. f1_score; the harmonic mean of precision and recall
4. Support; number of samples of the true response that lie in that class.
5. Recall; number of positive class predictions made out of all positive examples in the dataset.

In [ ]:
submission= pd.DataFrame(data=preds, index=list(y_test.index), columns=['predictions'])

filename = 'Hepatitis-C-submission.csv'
submission.to_csv(filename, index=False)

print('Saved file: ' + filename)